In [105]:
import pandas as pd
import numpy as np
import nltk
import string
from tqdm import tqdm



from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import train_test_split

In [106]:
tqdm.pandas()

In [107]:
input_path = 'data/input/'

In [108]:
all_train_data = pd.read_csv(input_path + 'train_data_prepped.csv').fillna('')
all_test_data = pd.read_csv(input_path + 'test_data_prepped.csv').fillna('')

In [109]:
all_train_data.head()

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized
0,0,anyway im getting of for a while,train,anyway im getting of for a while,anyway im getting of for a while,anyway im getting,anyway im get,anyway im get,anyway im getting
1,1,"my red, apache isn't feelin too well this morn...",train,my red apache isnt feelin too well this mornin...,my red apache isnt feelin too well this mornin...,red apache feelin well morning httpmypictmen,red apach feelin well morn httpmypictmen,red apach feelin wel morn httpmypictmen,red apache feelin well morning httpmypictmen
2,2,@danyelljoy you should be its great. friday w...,train,danyelljoy you should be its great friday wil...,danyelljoy you should be its great friday wil...,danyelljoy great friday great tooooooo,danyelljoy great friday great tooooooo,danyelljoy gre friday gre tooooooo,danyelljoy great friday great tooooooo
3,3,its 11:30pm and i dont wanna sleep; so i debat...,train,its 1130pm and i dont wanna sleep so i debated...,its pm and i dont wanna sleep so i debated wit...,pm wanna sleep debated end decided perfect tim...,pm wanna sleep debat end decid perfect time ba...,pm wann sleep deb end decid perfect tim bake! kid,pm wanna sleep debated end decided perfect tim...
4,4,why does twitter eat my dm's? not happy,train,why does twitter eat my dms? not happy,why does twitter eat my dms? not happy,twitter eat dms? happy,twitter eat dms? happi,twit eat dms? happy,twitter eat dms? happy


In [110]:
all_test_data.head()

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized
0,0,so cut... missed out on all the tce extreme me...,test,so cut missed out on all the tce extreme merch...,so cut missed out on all the tce extreme merch...,cut missed tce extreme merch! sizes,cut miss tce extrem merch! size,cut miss tce extrem merch! siz,cut missed tce extreme merch! size
1,1,thank you very much wasin this is awesome lol,test,thank you very much wasin this is awesome lol,thank you very much wasin this is awesome lol,thank much wasin awesome lol,thank much wasin awesom lol,thank much wasin awesom lol,thank much wasin awesome lol
2,2,squeezing in some internet time before dustin ...,test,squeezing in some internet time before dustin ...,squeezing in some internet time before dustin ...,squeezing internet time dustin goes work,squeez internet time dustin goe work,squeez internet tim dustin goe work,squeezing internet time dustin go work
3,3,@fanoou; oh i do have moral but i'm gonna be b...,test,fanoou oh i do have moral but im gonna be bigg...,fanoou oh i do have moral but im gonna be bigg...,fanoou oh moral im gonna bigger eat mc donalds,fanoou oh moral im gonna bigger eat mc donald,fanoou oh mor im gonn big eat mc donald,fanoou oh moral im gonna bigger eat mc donalds
4,4,@nsuissa i just bought sammy some joint/hip ...,test,nsuissa i just bought sammy some jointhip su...,nsuissa i just bought sammy some jointhip su...,nsuissa bought sammy jointhip supplements back...,nsuissa bought sammi jointhip supplement back ...,nsuiss bought sammy jointhip suppl back leg at...,nsuissa bought sammy jointhip supplement back ...


In [111]:
all_train_data[all_train_data.isnull().any(axis=1)]

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized


In [112]:
all_test_data[all_test_data.isnull().any(axis=1)]

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized


load train targets, binarize, drop nuetrals

In [113]:
train_target = pd.read_csv(input_path + 'train_results.csv')
train_target['target'].value_counts()

positive    520436
negative    519803
neutral         84
Name: target, dtype: int64

In [114]:
train_target.head()

,id,target
0,0,positive
1,1,negative
2,2,positive
3,3,positive
4,4,negative


In [115]:
all_train_data['target'] = train_results['target']

drop nuetral target rows to keep it binary

In [116]:
# all_train_data = all_train_data[all_train_data['target'] != 'neutral']
# all_train_data['target'] = [1 if t=='positive' else 0 for t in all_train_data['target'].values]
all_train_data['target'].value_counts()


positive    520436
negative    519803
neutral         84
Name: target, dtype: int64

### prepare bag of words<br>
we need a different model for each text format

Naive Bayes doesn't have any hyperparameters, so we'll test different versions of the text processing with different versions of naive bayes and corpos generation

the following parameters will be tested<br>
1. size of corpus vocab
2. corpus ngram varieties
3. bernoulli, multinomial or gaussian naive bayes
4. stemming/lemmatizing methods

In [12]:
vocab_sizes = [100, 1_000, 10_000, None]
ngrams = [(1, 1), (2, 2), (1, 2), (1, 3)]
models = [BernoulliNB(), GaussianNB(), MultinomialNB()]
text_columns = ['text_no_numerals', 'text_no_sw', 'text_porter_stemmed', 'text_lancaster_stemmed', 'text_lemmatized']
threshold = 0.5

# shuffle train data
all_train_data = all_train_data.sample(len(all_train_data))

# get data and target
train_X = all_train_data.iloc[:, :-1]
train_y  = all_train_data.iloc[:, -1]


# get 5000 data points for first round of training
train_v1 = train_X[:5000]
targets_v1 = train_y[:5000]

In [13]:
targets_v1.head()

917938    0
614316    0
803914    0
968913    0
745785    0
Name: target, dtype: int64

In [14]:
def trainModels(df, targets, vocab_sizes, models, text_columns, ngrams, threshold):

    param_tracker = []
    counter = 1
    for size in tqdm(vocab_sizes):
#         print(f'vocab size: {size}')

        for model in tqdm(models):
#             print(f'model type: {model}')

            for col in tqdm(text_columns):
#                 print(f'processing method: {col}')


                for ng in tqdm(ngrams):
#                     print(f'ngram range: {ng}')

                    counter+=1
                    if counter % 10==0:
                        print(f'{counter}')
                    vectorizer = CountVectorizer(max_features=size, ngram_range=ng)

#                     print('creating bag of words')
                    BOW = vectorizer.fit_transform(df[col])
                    BOW_array = BOW.toarray()

                    X_train, X_valid, y_train, y_valid = train_test_split(BOW_array, targets, test_size=0.2, random_state=42)
#                     print('fitting data')      
                    model.fit(X_train, y_train)


                    prob_predictions = model.predict_proba(X_valid)

                    # gets index of 1 column in prob_predictions
                    pos_predictions = [pred[list(model.classes_).index(1)] for pred in prob_predictions]

                    num_correct_pred = 0
                    for pred, actual in zip(pos_predictions, y_valid):
                        if pred >= threshold:
                            binary_pred = 1
                        else:
                            binary_pred = 0

                        if binary_pred==actual:
                            num_correct_pred+=1
                    accuracy = num_correct_pred / len(y_valid)
#                     print(f'accuracy: {accuracy}')
#                     print('\n\n')

                    param_tracker.append((size, str(model), col, ng, accuracy))
                    
    param_df = pd.DataFrame(param_tracker, columns = ['vocab_size', 'model_type', 'text_column', 'ngram', 'accuracy'])
    param_df = param_df.sort_values(by='accuracy', ascending=False)
    return param_df

                    
                
    # for vocab size we'll test

In [17]:
param_df = trainModels(train_v1, targets_v1, vocab_sizes, models, text_columns, ngrams, threshold)

  0%|                                                                                             | 0/3 [00:00<?, ?it/s]

  0%|                                                                                             | 0/5 [00:00<?, ?it/s]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 25%|█████████████████████▎                                                               | 1/4 [00:00<00:00,  5.92it/s]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:00<00:00,  5.38it/s]


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:00<00:00,  4.59it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.74it/s]


 20%|█████████████████                                                                    | 1/5 [00:01<00:04,  1.07s/it]


  0%|            

10





 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:00<00:00,  6.99it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.76it/s]


 60%|███████████████████████████████████████████████████                                  | 3/5 [00:02<00:01,  1.25it/s]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:00<00:00,  9.19it/s]


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:00<00:00,  7.17it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.86it/s]


 80%|████████████████████████████████████████████████████████████████████                 | 4/5 [00:03<00:00,  1.32it/s]


  0%|        

20





100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.72it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.28it/s]

 33%|████████████████████████████▎                                                        | 1/3 [00:03<00:07,  3.92s/it]

  0%|                                                                                             | 0/5 [00:00<?, ?it/s]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 25%|█████████████████████▎                                                               | 1/4 [00:00<00:00,  8.50it/s]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:00<00:00,  5.88it/s]


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:00<00:00,  4.75it/s]


100%|██████████

...




 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:14<00:05,  5.26s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:26<00:00,  6.67s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:25<00:00, 29.13s/it]

 67%|████████████████████████████████████████████████████████                            | 2/3 [05:30<02:41, 161.89s/it]

  0%|                                                                                             | 0/5 [00:00<?, ?it/s]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 25%|█████████████████████▎                                                               | 1/4 [00:02<00:06,  2.16s/it]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:08<00:09,  4.82s/it]


 75%|██████████

230





 25%|█████████████████████▎                                                               | 1/4 [00:01<00:05,  1.96s/it]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:06<00:07,  3.73s/it]


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:13<00:05,  5.19s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:25<00:00,  6.46s/it]


 60%|███████████████████████████████████████████████████                                  | 3/5 [01:29<00:57, 28.67s/it]


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]


 25%|█████████████████████▎                                                               | 1/4 [00:01<00:05,  1.84s/it]


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:06<00:07,  3.70s/it]


 75%|████████

240





 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:14<00:05,  5.29s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:26<00:00,  6.58s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:21<00:00, 28.30s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [10:20<00:00, 155.10s/it]


In [18]:
param_df[:20]

,vocab_size,model_type,text_column,ngram,accuracy
222,NaN,MultinomialNB(),text_no_numerals,"(1, 2)",0.722
104,1000.0,MultinomialNB(),text_no_sw,"(1, 1)",0.718
160,10000.0,MultinomialNB(),text_no_numerals,"(1, 1)",0.718
100,1000.0,MultinomialNB(),text_no_numerals,"(1, 1)",0.718
220,NaN,MultinomialNB(),text_no_numerals,"(1, 1)",0.716
180,NaN,BernoulliNB(),text_no_numerals,"(1, 1)",0.716
171,10000.0,MultinomialNB(),text_porter_stemmed,"(1, 3)",0.715
170,10000.0,MultinomialNB(),text_porter_stemmed,"(1, 2)",0.714
108,1000.0,MultinomialNB(),text_porter_stemmed,"(1, 1)",0.714
182,NaN,BernoulliNB(),text_no_numerals,"(1, 2)",0.714


In [20]:
param_df.to_csv('data/output/naive_bayes_params.csv', index=False)

In [15]:
param_df = pd.read_csv('data/output/naive_bayes_params.csv')

# ngrams get saved as string, convert back to int tuple
param_df['ngram'] = [(int(ng[1]), int(ng[4])) for ng in param_df['ngram'].values]

based on these results, we'll train the full dataset on these 20 combinations

In [16]:
param_df = param_df.replace({np.nan: None})

In [17]:
top_params = param_df[:10]

In [18]:
num_points = 100_000
num_train = int(num_points*0.8)

In [ ]:

model_accuracy = []

for i, param_set in enumerate(tqdm(top_params.values)):
    vocab_size, model_type, text_column, ngram, *_ = param_set
    print(vocab_size, model_type, text_column, ngram,)
    
    print('splitting train-validation data')
    
    # train is all training data with targets included!
    # sample to get smaller amount of data to deal with RAM constraints
    train_sample = all_train_data.sample(num_points)

    # get data and target
    sample_train_X = train_sample.iloc[:, :-1]
    sample_train_y  = train_sample.iloc[:, -1]


    

   
    
#     print(len(X_train), len(y_train), len(X_valid), len(y_valid))
    
    if vocab_size:
        vocab_size = int(vocab_size)
    
    if model_type == 'MultinomialNB()':
        model = MultinomialNB()
    elif model_type == 'BernoulliNB()':
        model = BernoulliNB()
    elif model_type == 'GaussianNB()':
        model = GaussianNB()
    else:
        print('warning, unrecognized model!')
        print(i, param_set)
        
    print('vectorizing')
    vectorizer = CountVectorizer(max_features=vocab_size, ngram_range=ngram)

    print('creating BOW')
    
    BOW = vectorizer.fit_transform(sample_train_X[text_column])
    BOW_array = BOW.toarray()
    

    
    
    # all sample_train_X and sample_train_y are the X, y sample from all of the training data, 
    #+ sample size determined by num_points
    X_train, X_valid, y_train, y_valid = train_test_split(BOW_array, sample_train_y, test_size=0.2, random_state=42)
    
    print('fitting model')
    model.fit(X_train, y_train)

    print('getting predictions')
    prob_predictions = model.predict_proba(X_valid)

    # gets index of 1 column in prob_predictions
    pos_predictions = [pred[list(model.classes_).index(1)] for pred in prob_predictions]
    
    
    # get accuracy
    num_correct_pred = 0
    for pred, actual in zip(pos_predictions, y_valid):
        if pred >= threshold:
            binary_pred = 1
        else:
            binary_pred = 0

        if binary_pred==actual:
            num_correct_pred+=1
    accuracy = num_correct_pred / len(y_valid)
    print(accuracy)
    print('\n')
    
    model_accuracy.append(accuracy)
    
    

  0%|                                                                                            | 0/10 [00:00<?, ?it/s]

None MultinomialNB() text_no_numerals (1, 2)
splitting train-validation data
vectorizing
creating BOW


In [ ]:
top_params['final_validation_acc'] = model_accuracy

In [ ]:
top_params.to_csv('top_naive_bayes.csv', index=False)

## train final model with best parameters

In [117]:
# 1000.0	MultinomialNB()	text_no_sw	(1, 1)	

all_train_data = all_train_data.sample(len(all_train_data))




vectorizer = CountVectorizer(max_features=1000, ngram_range=(1, 1))








In [118]:
train_BOW = vectorizer.fit_transform(all_train_data['text_no_sw'])
train_BOW_array = train_BOW.toarray()



In [119]:
test_BOW = vectorizer.transform(test['text_no_sw'])
test_BOW_array = test_BOW.toarray()

In [120]:
# get data and target
train_X = all_train_data.iloc[:, :-1]
train_y  = all_train_data.iloc[:, -1]

In [121]:
model = MultinomialNB()
model.fit(train_BOW_array, train_y)

MultinomialNB()

In [122]:
model.classes_

array(['negative', 'neutral', 'positive'], dtype='<U8')

In [123]:
predictions = model.predict(test_BOW_array)

# # gets index of 1 column in prob_predictions
# pos_predictions = [pred[list(model.classes_).index(1)] for pred in prob_predictions]


# # get accuracy
# num_correct_pred = 0
# for pred, actual in zip(pos_predictions, y_valid):
#     if pred >= threshold:
#         binary_pred = 1
#     else:
#         binary_pred = 0

#     if binary_pred==actual:
#         num_correct_pred+=1
# accuracy = num_correct_pred / len(y_valid)
# print(accuracy)
# print('\n')

In [124]:
pdf = pd.DataFrame(predictions).reset_index()
pdf.columns  = ['id', 'target']


In [130]:
pdf['target'].value_counts()

0    288066
2    272104
1         5
Name: target, dtype: int64

In [133]:
pdf['target'] = [0 if t == 'negative' else 2 if t=='positive' else 2 for t in pdf['target'].values]

In [134]:
pdf.to_csv('data/output/first_pred.csv', index=False)